In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("data.csv")

This is *an* evolution of preferences, produced with the model initiated with a map of width $N = 300$, that is, having $90000$ voters, having uniformly distributed initial preferences, carried out $300$ generations.

In [4]:
data

,Unnamed: 0,E,D,C,B,A,O,A',B',C',D',E',outcomes,str_out_det
0,0,8206,8289,8250,8072,8188,8178,8136,8141,8167,8290,8083,"{'tie': 0, 'left_suc_strategic': 0, 'left_unsu...","{'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -..."
1,1,13272,7547,7153,6863,6385,8178,6401,6547,7015,7315,13324,"{'tie': 826, 'left_suc_strategic': 1463, 'left...","{'left_suc_strategic': {-9: 0, -7: 165, -5: 27..."
2,2,15053,7209,6718,6447,5926,8178,5691,5981,6648,7007,15142,"{'tie': 664, 'left_suc_strategic': 959, 'left_...","{'left_suc_strategic': {-9: 0, -7: 103, -5: 22..."
3,3,15501,7161,6652,6349,5738,8178,5545,5835,6524,6861,15656,"{'tie': 449, 'left_suc_strategic': 602, 'left_...","{'left_suc_strategic': {-9: 0, -7: 73, -5: 127..."
4,4,15618,7174,6591,6364,5684,8178,5503,5909,6512,6849,15618,"{'tie': 368, 'left_suc_strategic': 375, 'left_...","{'left_suc_strategic': {-9: 0, -7: 38, -5: 87,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,10948,8098,7610,7359,6714,8178,6881,7244,7792,8138,11038,"{'tie': 55, 'left_suc_strategic': 0, 'left_uns...","{'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -..."
296,296,10948,8098,7610,7359,6714,8178,6881,7244,7792,8138,11038,"{'tie': 55, 'left_suc_strategic': 0, 'left_uns...","{'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -..."
297,297,10948,8098,7610,7359,6714,8178,6881,7244,7792,8138,11038,"{'tie': 55, 'left_suc_strategic': 0, 'left_uns...","{'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -..."
298,298,10948,8098,7610,7359,6714,8178,6881,7244,7792,8138,11038,"{'tie': 55, 'left_suc_strategic': 0, 'left_uns...","{'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -..."


# Distribution of the strategic votes

In [7]:
data.str_out_det

0      {'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -...
1      {'left_suc_strategic': {-9: 0, -7: 165, -5: 27...
2      {'left_suc_strategic': {-9: 0, -7: 103, -5: 22...
3      {'left_suc_strategic': {-9: 0, -7: 73, -5: 127...
4      {'left_suc_strategic': {-9: 0, -7: 38, -5: 87,...
                             ...                        
295    {'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -...
296    {'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -...
297    {'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -...
298    {'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -...
299    {'left_suc_strategic': {-9: 0, -7: 0, -5: 0, -...
Name: str_out_det, Length: 300, dtype: object

In [8]:
# Let's print out some random generation to see what it looks like.

In [9]:
data.str_out_det[10]

"{'left_suc_strategic': {-9: 0, -7: 15, -5: 37, -3: 43, -1: 35}, 'left_unsuc_best_try': {-9: 1225, -7: 163, -5: 143, -3: 130, -1: 131}, 'right_suc_strategic': {1: 32, 3: 47, 5: 18, 7: 18, 9: 0}, 'right_unsuc_best_try': {1: 129, 3: 135, 5: 144, 7: 130, 9: 1257}}"

In [13]:
# We can, for example, look at the total number of strategic votes of each types.